# Fabric Sync Notebook

#### Fabric Sync Environment

This notebook is configured to use the required libraries from a Fabric Spark Environment.

In [ ]:
from FabricSync.BQ.Sync import BQSync
from FabricSync.BQ.Enum import SyncScheduleType

#### BQ Spark Connector Optimizations

In [ ]:
spark.conf.set("readSessionCacheDurationMins", "1")
spark.conf.set("preferredMinParallelism", 600) #This varies based on size of data and compute environment size
spark.conf.set("responseCompressionCodec", "RESPONSE_COMPRESSION_CODEC_LZ4")
spark.conf.set("bqChannelPoolSize", 80) #Match the number of executor cores for your configuration

#For big data loads, set so that the BQ connection does not timeout
spark.conf.set("httpConnectTimeout", 0)
spark.conf.set("httpReadTimeout", 0)

#### Config
The set-up process creates a minimal config file based on the parameters provided. 

Note: If you upload to a OneLake destination, it must be in the default Lakehouse and the <code>config_json_path</code> should point to the File API path (example: <code>/lakehouse/default/Files/myconfigfile.json</code>).

In [ ]:
config_json_path = "<<<PATH_TO_USER_CONFIG>>>"
schedule_type = SyncScheduleType.AUTO
optimize_metadata = False
credential_provider = notebookutils.credentials

#### Running Fabric Sync

In [ ]:
bq_sync = BQSync(config_json_path, credential_provider)
metadata_result = bq_sync.sync_metadata()

Before you continue, carefully evaluate your config for correctness.

Once you run the next step, your load configuration is locked and cannot be changed without manually resetting the sync metadata and sync'd data.

In [ ]:
if metadata_result:
    sync_result = bq_sync.run_schedule(schedule_type=schedule_type, sync_metadata=False, optimize_metadata=optimize_metadata)